In [1]:
n = 2**2
q = random_prime(n**8, lbound=n**7)
k = 0
while 2 ** k < q:
    k += 1
    
Zx.<x> = ZZ[]

print(f"n={n}")
print(f"q={q}")
print(f"k={k}")

n=4
q=30977
k=15


In [2]:
def invert_mod_q(f):
    # f in ZZ[x], f^(-1) in ZZ[x]
    # ((f * f^(-1)) % (x^n + 1)) % q = 1
    
    T = Zx.change_ring(Integers(q)).quotient(x^n+1)
    return Zx(lift(1/T(f)))

In [3]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler
from math import sqrt

def generate_trapdoor():
    # generate trapdoor matrix B, list[[]] representation
    magic_constant = [k * (0.1 + 0.002 * i) for i in range(k)]

    B = [[None for j in range(k + 1)] for i in range(k + 1)]

    eta = 1
    for i in range(k):
        for j in range(k + 1):
            sigma = magic_constant[i] * (q ** (1 / (k + 1)))
            sig = sqrt(1 / (n * (k + 1 - i))) * sigma
            B[i][j] = DiscreteGaussianDistributionPolynomialSampler(Zx, n, sig)()
    
    return B

In [4]:
def submatrix(M, row, col):
    nrow = len(M)
    ncol = len(M[0])
    rep = [[M[i][j] for j in range(ncol) if j != col] for i in range(nrow) if i != row]
    return rep

In [5]:
def matrix_invert_mod_q(f):
    # matrix: f in ZZ[x], f^(-1) in ZZ[x]
    # ((f * f^(-1)) % (x^n+1)) % q = 1
    
    f_det = f.determinant()
    f_det_inv = invert_mod_q(f_det)
    f_adj = f.adjugate()
    f_inv = f_adj * f_det_inv
    return f_det, f_adj, f_inv

In [6]:
def neg_mat(M):
    # M in [[]]
    # calculate the negative of a matrix
    return [[-M[i][j] for j in range(k)] for i in range(k)]

In [40]:
B = generate_trapdoor()

In [41]:
f = matrix(Zx, neg_mat(submatrix(B, k, 0)))

In [42]:
f_det, f_adj, f_inv = matrix_invert_mod_q(f)

In [43]:
((f_inv * f)% (x^n+1)) % q

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]

In [44]:
g = matrix(Zx, k, 1, [B[j][0] for j in range(k)])

In [45]:
h = ((f_inv * g) % (x^n + 1)) % q  # h = f^(-1) * g mod q

In [46]:
g_adjf = Zx(list((f_adj * g) % (x^n + 1))[0][0])

In [47]:
def xgcd_mod(f, g):
    df, uf, vf = (f % (x^n + 1)).xgcd(x^n + 1)
    dg, ug, vg = (g % (x^n + 1)).xgcd(x^n + 1)
    return df, dg, uf, ug

In [48]:
xgcd_mod(f_det, g_adjf)

(131755222727167512938929,
 224132819068681139665296,
 166650810641792916*x^3 + 142593438782008252*x^2 - 2326140005957730*x + 58431914971302191,
 236832300623487816*x^3 + 130030750509637104*x^2 - 70247868910310160*x + 210711712687962336)

In [49]:
rf, rg, _pf, _pg = xgcd_mod(f_det, g_adjf)

In [50]:
gcd(rf, rg)

1

In [51]:
gcd(rf, q)

1

In [52]:
def to_bar(f):
    return Zx([f[0]]+[-f[n - i] for i in range(1, n)])

In [53]:
def cal_FG(f, g):
    rf, rg, _pf, _pg = xgcd_mod(f, g)
    d, u, v = rf.xgcd(rg)
    F = (q * v * (-1) * _pg) % (x^n + 1)
    G = (q * u * _pf) % (x^n + 1)
    a = (f * to_bar(f) + g * to_bar(g)) % (x^n + 1)
    b = (F * to_bar(f) + G * to_bar(g)) % (x^n + 1)
#     k = Zx([int(list(b)[i]/list(a)[i]) for i in range(n)])
    return F, G

In [54]:
F, G = cal_FG(f_det, g_adjf)

In [55]:
ffgg = (f_det * to_bar(f_det) + g_adjf * to_bar(g_adjf)) % (x^n + 1)

In [56]:
fFgG = (F * to_bar(f_det) + G * to_bar(g_adjf)) % (x^n + 1)

In [57]:
dfg, ufg, vfg = ffgg.xgcd(x^n + 1)

In [58]:
def convert_to_Zx(f):
    return Zx([int(i) for i in f])

In [59]:
def round_(f, num):
    # f in Zx
    return Zx([round(list(f)[i]/num) for i in range(n)])

In [60]:
k_fg = round_(convert_to_Zx((ufg * fFgG) % (x^n + 1)), int(dfg))

In [61]:
F

114616506216232400033269522681478744402127168*x^3 + 62929213138805761658153173979567881562308992*x^2 - 33996905331067539991689335666250200466007680*x + 101975280667174475493390067617856600518544128

In [62]:
G

137199104526712158478572851378230647026806740*x^3 + 117393321022165326427278865842929392440134780*x^2 - 1915048145233120126603168713110776046568450*x + 48105415023004879448250918118688795433739615

In [63]:
(f_det * G - g_adjf * F) % (x^n + 1) # f * G - g * F = q

30977

In [64]:
# Reduce F and G, F = F - k * f, G = G - k * g
F_ = F - (k_fg * (f_det % (x^n + 1))) % (x^n + 1)
G_ = G - (k_fg * (g_adjf % (x^n + 1))) % (x^n + 1)

In [65]:
(f_det * G_ - g_adjf * F_) % (x^n + 1) # f * G - g * F = q, after reduce

30977

In [66]:
# insert F, G into trapdoor matrix B
B[k][0] = G_
B[k][1] = -F_
for i in range(2, k + 1):
    B[k][i] = Zx([0])

In [67]:
B = matrix(Zx, k + 1, k + 1, B)

In [68]:
# A = [1 h]^T
A = [None for j in range(k + 1)]
for j in range(1, k + 1):
    A[j] = h[j - 1][0]
A[0] = Zx([1])

In [69]:
A = matrix(Zx, k + 1, 1, A)

In [72]:
((B * A) % (x^n + 1)) % q

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]

In [74]:
(B.determinant()) % (x^n + 1)

30977